In [ ]:
!pip install -q langchain-groq langchain chromadb sentence-transformers unstructured
!pip install -U langchain-community
!pip install -U langchain
!pip install -U httpx
!pip install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 27.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 97.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from langchain_groq import ChatGroq
import os

# Setup for Groq API key
os.environ["GROQ_API_KEY"] = "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"
groq_api_key = os.getenv("GROQ_API_KEY")

# Initialize LLM with low temperature
llm = ChatGroq(
    api_key=groq_api_key,
    model_name="llama3-8b-8192",
    temperature=0.2  # 🔽 Lower = more consistent, deterministic
)


In [ ]:
!pip install pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 86.5 MB/s eta 0:00:00


In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

# --- Configuration ---
pdf_file_path = "/content/drive/MyDrive/RS775-4538_Datasheet.pdf"
persist_directory = "/content/drive/MyDrive/RS775-4538_Datasheet_DB_RS775"
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"

# --- Embedding Setup ---
embedding = HuggingFaceEmbeddings(model_name=embedding_model)

# --- Load PDF ---
loader = PyMuPDFLoader(pdf_file_path)
documents = loader.load()

# --- Split Text into Chunks ---
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(documents)

print(f"✅ Chunks created from RS775 datasheet: {len(chunks)}")

# --- Store in Chroma Vector DB ---
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding,
    persist_directory=persist_directory
)
vectorstore.persist()

print("✅ RS775 Datasheet Vector DB created and saved.")


✅ Chunks created from RS775 datasheet: 2
✅ RS775 Datasheet Vector DB created and saved.


<ipython-input-7-3111249558>:31: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [ ]:
import shutil
from google.colab import files

# Path to your Chroma DB directory
persist_directory = "/content/drive/MyDrive/RS775-4538_Datasheet_DB_RS775"

# Output zip file name
zip_filename = "RS775_VectorDB.zip"

# Create zip file
shutil.make_archive("RS775_VectorDB", 'zip', persist_directory)

# Download zip file in Colab
files.download(zip_filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from google.colab import files
import zipfile
import os

# 1️⃣ Upload the zipped vector DB (uploaded manually by user)
uploaded = files.upload()

# 2️⃣ Extract the uploaded zip file
zip_name = list(uploaded.keys())[0]
unzip_path = "/content/uploaded_vectordb"

with zipfile.ZipFile(zip_name, 'r') as zip_ref:
    zip_ref.extractall(unzip_path)

# 3️⃣ Load the embedding model
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 4️⃣ Load vectorstore from unzipped path
vectorstore = Chroma(
    persist_directory=unzip_path,
    embedding_function=embedding
)

# 5️⃣ Create retriever
retriever = vectorstore.as_retriever()

print("✅ Vector DB loaded successfully from uploaded zip.")


Saving RS775_VectorDB.zip to RS775_VectorDB.zip


<ipython-input-4-352212808>:18: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datas

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-4-352212808>:21: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


✅ Vector DB loaded successfully from uploaded zip.


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# ✅ Updated prompt template for datasheet-based answers
prompt_template = PromptTemplate(
    input_variables=["question", "context"],
    template=(
        "You are an AI assistant that helps engineers understand technical information from motor datasheets. "
        "Use the provided context to answer the question accurately. Stick to the content of the RS775-4538 motor datasheet. "
        "If the answer is not available in the context, say you don't know.\n\n"
        "Context:\n{context}\n\n"
        "Question: {question}\n\n"
        "Answer:"
    )
)

# ✅ Initialize the LLM chain using the updated prompt
qa_chain = LLMChain(
    llm=llm,  # e.g., OpenAI model or HuggingFace LLM
    prompt=prompt_template
)

# ✅ Function to answer datasheet questions using RAG
def answer_motor_datasheet_question_with_rag(question):
    # Retrieve the relevant chunks from Chroma
    retrieved_docs = retriever.get_relevant_documents(question)

    # Combine content from retrieved documents
    context = " ".join([doc.page_content for doc in retrieved_docs])

    # Run the LLM chain with both context and question
    response = qa_chain.run(question=question, context=context)

    return response


<ipython-input-5-670262687>:18: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  qa_chain = LLMChain(


In [ ]:
response = answer_motor_datasheet_question_with_rag("What is the stall torque of the RS775 motor?")
print(response)


<ipython-input-5-670262687>:26: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(question)
<ipython-input-5-670262687>:32: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(question=question, context=context)


According to the datasheet, the stall torque of the RS775-4538 DC Motor is approximately 2500 mN.m.


In [ ]:
response = answer_motor_datasheet_question_with_rag("What is the maximum continuous current the motor can handle safely?")
print(response)

According to the datasheet, the Rated Current is 10 A, which is the maximum continuous current the motor can handle safely.


In [ ]:
response = answer_motor_datasheet_question_with_rag("What are the recommended operating practices?")
print(response)

According to the RS775-4538 DC Motor Datasheet, the recommended operating practices are:

* Use current limiting driver circuits.
* Ensure proper mounting and ventilation.
* Avoid long stalling and overcurrent.
* Use flyback diodes for switching.
* Periodically inspect for brush wear.

These recommendations are listed in section 5 of the datasheet, titled "Recommended Operating Practices".


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 🧠 Prompt Template for Generating Motor Issue Recommendations
prompt_template = PromptTemplate(
    input_variables=["issue", "context"],
    template=(
        "You are an AI assistant with expertise in electrical engineering and motor systems. "
        "Based on the following datasheet context for the RS775-4538 DC motor and the issue described, "
        "provide a clear, practical recommendation for resolving or mitigating the issue. "
        "Base your reasoning on the datasheet when possible. If not covered, use standard best practices. "
        "Do NOT invent technical specs that are not in the datasheet.\n\n"
        "Datasheet Context:\n{context}\n\n"
        "Observed Issue:\n{issue}\n\n"
        "Recommended Action:"
    )
)


In [ ]:
# ✅ Initialize the LLM chain with the updated recommendation prompt
recommendation_chain = LLMChain(
    llm=llm,  # HuggingFaceHub, ChatOpenAI, etc.
    prompt=prompt_template
)


In [ ]:
# Function that generates a recommendation based on an alert/issue
def get_motor_issue_recommendation(issue_description):
    # Retrieve relevant chunks from vector DB using the issue
    retrieved_docs = retriever.get_relevant_documents(issue_description)

    # Merge retrieved datasheet content into a single context string
    context = " ".join([doc.page_content for doc in retrieved_docs])

    # Generate a recommendation
    recommendation = recommendation_chain.run(issue=issue_description, context=context)

    return recommendation


In [ ]:
issue = "Overheating: temperature 72.85°C exceeds 70.0°C"
print(get_motor_issue_recommendation(issue))


Based on the datasheet and the observed issue of overheating, I recommend the following actions to resolve or mitigate the issue:

1. **Improve Ventilation**: Ensure proper mounting and ventilation as recommended in the datasheet (Section 5). Make sure the motor is installed in a well-ventilated area, and the surrounding environment is not obstructing airflow. This will help to dissipate heat generated by the motor.
2. **Reduce Stall Time**: Avoid long stalling and overcurrent as recommended in the datasheet (Section 5). If the motor is stalling frequently, it may be causing excessive heat buildup. Implement measures to reduce stall time, such as adjusting the motor's control algorithm or adding a soft-start feature.
3. **Monitor and Limit Current**: Use current limiting driver circuits as recommended in the datasheet (Section 5). This will help prevent overcurrent conditions that can cause overheating. Implement current limiting or overcurrent protection in the motor control circuitry

In [ ]:
issue = "Voltage drop: V=19.6V below 20.0V"
print(get_motor_issue_recommendation(issue))

Based on the datasheet and the observed issue, I recommend the following action to resolve or mitigate the voltage drop:

**Recommendation:** Check the motor's mounting and ventilation to ensure proper airflow and heat dissipation. The datasheet recommends "Ensure proper mounting and ventilation" in the Recommended Operating Practices section.

**Rationale:** The voltage drop observed (V=19.6V below 20.0V) may be indicative of excessive heat generation within the motor, which can cause a decrease in voltage. As the motor operates, it generates heat, and if the heat is not dissipated properly, it can lead to a voltage drop. Proper mounting and ventilation can help to dissipate heat efficiently, ensuring the motor operates within its specified parameters.

**Additional Recommendations:**

1. Verify that the motor is properly mounted and secured to the surrounding structure to prevent excessive vibration or movement, which can also contribute to heat generation.
2. Check the motor's surro

In [ ]:
issue = "Current spike: ΔI=2.20A (I=2.49A)"
print(get_motor_issue_recommendation(issue))

Based on the datasheet and the observed issue, I recommend the following action to resolve or mitigate the current spike:

**Recommendation:** Implement a current limiting driver circuit to regulate the motor current and prevent excessive current draw.

**Rationale:**

1. The datasheet recommends using current limiting driver circuits (Section 5, Recommended Operating Practices). This suggests that the manufacturer has identified current limiting as a critical aspect of motor operation.
2. The observed current spike (ΔI=2.20A, I=2.49A) exceeds the rated current (10 A) and stall current (~80 A) specified in the datasheet. This indicates that the motor is operating outside its designed parameters, which can lead to premature wear, overheating, or even damage.
3. By implementing a current limiting driver circuit, you can ensure that the motor current remains within the recommended range, preventing excessive current draw and reducing the risk of damage or overheating.

**Additional Recomm

In [ ]:
issue = "Efficiency low: 73.9%"
print(get_motor_issue_recommendation(issue))

Based on the datasheet and the observed issue of low efficiency (73.9%), I recommend the following actions to mitigate or resolve the issue:

1. **Verify proper mounting and ventilation**: Ensure that the motor is properly mounted and ventilated to prevent overheating, which can reduce efficiency. Check that the motor is securely fastened to the mounting flange and that there is sufficient clearance around the motor for airflow.
2. **Check for brush wear**: As recommended in the datasheet, inspect the brushes for wear. Worn-out brushes can reduce efficiency and motor performance. If the brushes are worn out, replace them with new ones.
3. **Optimize the motor's operating point**: The datasheet provides a performance curve, which can help identify the optimal operating point for the motor. Check the curve to ensure that the motor is operating within its recommended range. If the motor is operating at a low load or at a high speed, it may be operating outside its optimal range, which can

In [ ]:
issue = "Average power: 56.9 W, Peak power: 104.3 W"
print(get_motor_issue_recommendation(issue))


Based on the datasheet and the observed issue, I recommend the following:

**Action:** Reduce the peak power consumption to prevent overheating and potential damage to the motor.

**Reasoning:**

1. The datasheet recommends using current limiting driver circuits to avoid overcurrent. The observed peak power of 104.3 W is higher than the rated output power of 150 W, which may cause overheating and reduce the motor's lifespan.
2. The datasheet also recommends avoiding long stalling and overcurrent. The peak power consumption indicates that the motor is being stalled or overloaded, which can cause excessive heat generation and damage to the motor.
3. To mitigate this issue, I suggest implementing a current limiting driver circuit or a power controller that can regulate the motor's current and prevent overcurrent. This will help to reduce the peak power consumption and prevent overheating.

**Additional Recommendations:**

1. Ensure proper mounting and ventilation of the motor to prevent o

# Task
Provide instructions on how to run the current Colab notebook locally, including downloading the notebook, installing dependencies, setting up environment variables, adjusting file paths, and running the notebook.

## Download the notebook

### Subtask:
Download the current Colab notebook to your local machine.


## Install dependencies

### Subtask:
Install all the necessary libraries and dependencies locally using pip.


**Reasoning**:
Install all the required libraries using pip.



In [ ]:
!pip install langchain-groq langchain chromadb sentence-transformers unstructured langchain-community httpx groq pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 19.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━